In [1]:
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd
from shapely import wkt
import geopandas as gpd

/Users/kathrynpanger/Documents/censusMaps/venv/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_31725/2461549588.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to 

In [2]:
chicagoDf = pd.read_csv("../localData/censusVars.csv")
chicagoDf['geometry'] = chicagoDf['geometry'].apply(wkt.loads)
chicagoDf = gpd.GeoDataFrame(chicagoDf, crs='epsg:4326')

In [5]:
quantile = chicagoDf["total_population"].quantile(q = 0.75)
chicagoDf['pop_if_over_quantile'] = chicagoDf["total_population"].apply(lambda x: round(x,2) if x>quantile else " ")

In [ ]:
f, ax = plt.subplots(1,1,figsize=(20,20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
ax.set_facecolor('k')
# chicago.apply(lambda x: ax.annotate(text=x['population'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);
chicagoDf.plot('total_population', cmap='Greens', ax = ax, alpha=0.5, legend = True, cax = None, color = None) #add population
chicagoDf.apply(lambda x: ax.annotate(text=x['pop_if_over_quantile'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);#add population annotation if over quantile
#cx.add_basemap(ax, crs=chicago.crs.to_string(), source=cx.providers.OpenStreetMap.HOT, zoom=12, alpha = 0.5)
cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha = 1)

title = 'Total Population by Tract'
ax.set_title(title)
plt.savefig(f"../mapImages/{title.replace(' ', '')}.png")